***INSTALL REQUIREMENTS***

In [ ]:
# %pip install ultralytics==8.2.103 -q
# %pip install roboflow --quiet
# %pip install supervision==0.24.0 -q
# %pip install face_recognition opencv-python numpy
# %pip install face_recognition
# %pip install supervision

***IMPORT STATION***

In [ ]:
from ultralytics import YOLO
from IPython.display import display, Image
from IPython.display import clear_output
from datetime import datetime
from flask import Flask, render_template, Response
import yaml
import supervision as sv
import ultralytics
import cv2
import sqlite3
import os
import atexit
import numpy as np
import face_recognition
import time

# Path model dan database
MODEL_PATH = "runs/obb/train3/weights/best.pt"
DB_PATH = "presensi.db"
FACES_DIR = "/home/abe-tanu/Documents/Code/Python/Artificial-Intelligence/faces"

***TRAINING***

In [ ]:
!yolo settings sync=False

ultralytics.checks()

dataset = "/home/abe-tanu/Documents/Code/Python/Artificial-Intelligence/OBB AI PPTI 21 Team 2.v2i.yolov8-obb/datasets"

# Download the dataset
with open(f'{dataset}/data.yaml', 'r') as f:
    data = yaml.safe_load(f)
data['train'] = '../train/images'
data['val'] = '../valid/images'
data['test'] = '../test/images'
if 'path' in data:
  del data['path']
with open(f'{dataset}/data.yaml', 'w') as f:
    yaml.dump(data, f, sort_keys=False)

# Train the model
model = YOLO('yolov8n-obb.pt')
results = model.train(data=f"{dataset}/data.yaml", epochs=10, imgsz=640)


***MODELING***

**Face Encoding**

In [ ]:
# Load wajah dari folder
known_face_encodings = []
known_face_names = []
for filename in os.listdir(FACES_DIR):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        path = os.path.join(FACES_DIR, filename)
        image = face_recognition.load_image_file(path)
        encodings = face_recognition.face_encodings(image)
        if encodings:
            known_face_encodings.append(encodings[0])
            known_face_names.append(os.path.splitext(filename)[0])


**Utility Function**

In [ ]:
# Koneksi database
conn = sqlite3.connect(DB_PATH, check_same_thread=False)
c = conn.cursor()
c.execute('''
    CREATE TABLE IF NOT EXISTS presensi (
        nama TEXT,
        status TEXT,
        waktu TEXT
    )
''')
c.execute("DELETE FROM presensi")
conn.commit()

def sudah_terdeteksi(nama):
    c.execute("SELECT 1 FROM presensi WHERE nama=? AND status='Hadir'", (nama,))
    return c.fetchone() is not None

def tandai_hadir(nama):
    waktu = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    c.execute("INSERT INTO presensi (nama, status, waktu) VALUES (?, 'Hadir', ?)", (nama, waktu))
    conn.commit()

def get_status_presensi(daftar_anak):
    c.execute("SELECT nama, waktu FROM presensi WHERE status = 'Hadir'")
    hasil = c.fetchall()
    nama_hadir = set(nama for nama, _ in hasil)
    tidak_hadir = [nama for nama in daftar_anak if nama not in nama_hadir]
    return hasil, tidak_hadir

**Setup Yolo**

In [ ]:
#  Daftar anak-anak
daftar_anak = ["Adriel Bernhard T", "Jonea Kristiawan", "Kevin Tanwiputra", "Kevin Jiovanni Kuslin"]

# Load YOLOv8-OBB
model = YOLO(MODEL_PATH)

***Flask***

**Streaming Flask**

In [ ]:
app = Flask(__name__)
obb_annotator = sv.OrientedBoxAnnotator()

def gen():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("[ERROR] Kamera tidak bisa dibuka.")
        return

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    frame_count = 0
    results = None

    while True:
        try:
            success, frame = cap.read()
            if not success:
                print("[ERROR] Gagal membaca frame dari kamera")
                break
            else:
                print("[DEBUG] Frame berhasil dibaca")

            frame_count += 1
            annotated_frame = frame.copy()

            # === Bagian YOLOv8-OBB ===
            if frame_count % 3 == 0:
                start = time.time()
                results = model.predict(frame, conf=0.75)
                print(f"[INFO] YOLOv8 OBB took {time.time() - start:.2f}s")

            if results:
                for r in results:
                    print("[DEBUG] r.obb:", r.obb)
                    if r.obb is not None and hasattr(r.obb, "xyxy"):
                        classes = r.obb.cls.cpu().numpy().astype(int)
                        detections = sv.Detections(
                            xyxy=r.obb.xyxy.cpu().numpy(),
                            confidence=r.obb.conf.cpu().numpy(),
                            class_id=classes
                        )

                        labels = [model.names[c] for c in classes]
                        annotated_frame = obb_annotator.annotate(annotated_frame, detections)
                        for box, label in zip(detections.xyxy, labels):
                            x1, y1, x2, y2 = map(int, box)
                            cv2.putText(annotated_frame, label, (x1, y1 - 10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)


                        for cls in classes:
                            nama_terdeteksi = model.names[cls]
                            if nama_terdeteksi in daftar_anak and not sudah_terdeteksi(nama_terdeteksi):
                                tandai_hadir(nama_terdeteksi)
                                print(f"[INFO] {nama_terdeteksi} TERDETEKSI (OBB)")

            # === Face Recognition ===
            rgb_frame = annotated_frame[:, :, ::-1]
            face_locations = face_recognition.face_locations(rgb_frame)

            if not face_locations:
                continue

            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                name = "Unknown"
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]

                if name in daftar_anak and not sudah_terdeteksi(name):
                    tandai_hadir(name)
                    print(f"[INFO] {name} TERDETEKSI (Face)")

                cv2.rectangle(annotated_frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.putText(annotated_frame, name, (left, top - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

            # === Overlay presensi ===
            cv2.rectangle(annotated_frame, (10, 10), (500, 10 + 30 + 25 * len(daftar_anak)), (0, 0, 0), -1)
            cv2.putText(annotated_frame, "Presensi Siswa", (20, 35),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            y_offset = 60
            for nama in sorted(daftar_anak):
                status = "Hadir" if sudah_terdeteksi(nama) else "Belum Hadir"
                warna = (0, 255, 0) if status == "Hadir" else (0, 0, 255)
                cv2.putText(annotated_frame, f"{nama} - {status}", (20, y_offset),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.55, warna, 2)
                y_offset += 25

            # === Streaming frame ===
            _, buffer = cv2.imencode('.jpg', annotated_frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

        except Exception as e:
            print(f"[ERROR] Streaming error: {e}")
            continue

# ===== ROUTES FLASK =====
@app.route('/')
def index():
    return render_template("index.html")

@app.route('/video_feed')
def video_feed():
    return Response(gen(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/rekap')
def rekap():
    hadir, tidak_hadir = get_status_presensi(daftar_anak)
    return render_template("rekap.html", hadir=hadir, tidak_hadir=tidak_hadir)

**Run Flask**

In [ ]:
app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)